## Document Question Answering Notebook

## Setup: 

- Make sure you have installed the required packages (see requirements.txt)
- Make sure you have preprocessed the documents (see README.md)
- You also need a working internet connection to download the AI language models 
- You should have at least 8GB of RAM installed on your machine (Also a GPU is preferred for faster response generation)

## How to run: 
- Run Setup notebook cells for downloading the models and loading the preprocessed documents
- If you run the notebook for the first time, it will take a while to download the models so grab a cup of coffee :)
- The code will automatically decide which model to download based on if a GPU is available or not ( change the device parameter in the "load_inference_model" function to set manually )
- Either choose a predefined example query from the dropdown or enter your own query
- The current standard settings are to extract the 3 top snippets from the documents collection to base the model answer on it

## Visualisation:
- Visualise the output of the model and compare it with the extracted snippets from the documents
- The visualisation is currently highlighting the 5 top sentences that influenced the answer generation



### Setup: 

In [1]:
## automatic reload when imported files change
%load_ext autoreload
%autoreload 2

import sys 
sys.path.append('../')
import rag.rag_retrieval_utils as rag_utils
import rag.rag_visualisation as rag_visualisation
import rag.rag_llm_classes as rag_llm_classes
import pandas as pd 
from sentence_transformers import SentenceTransformer
import json 


/home/paul/miniconda3/envs/llm_rag/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /home/paul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/paul/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/paul/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/paul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.1-8B-Instruct')

In [6]:
tokenizer.eos_token_id

128009

In [2]:
## Load embedding model 
model_emb = SentenceTransformer('BAAI/bge-large-en-v1.5')

## Load LLM model
## either device == 'cpu' or 'cuda' (for GPU) or None for self-selection
## specify specific model with link to huggingface repository:
## e.g., model_name='QuantFactory/Meta-Llama-3.1-8B-Instruct-GGUF', file name="Meta-Llama-3.1-8B-Instruct.Q5_K_M.gguf" for cpu model
## OR only model_name = Nexusflow/Starling-LM-7B-beta for GPU models 
## check for new models https://arena.lmsys.org/
## otherwise, loads standard model which is llama-3
## Note: Might need to adjust tokenizer_dict in rag_llm_classes.load_cpu_model
## Note some models needs a license agreement before using
llm = rag_llm_classes.load_inference_model(device='cpu')

## Load document chunks 
input_path = "../datasets/chunked_documents/nasa_teaching_spacesuit/chunking_cks_1536_ovl_8.json"
docs = rag_utils.load_data(input_path) 

## drop duplicates according to the text column
data = pd.DataFrame(docs)
data.drop_duplicates(subset=['text'], inplace=True)
docs = data.to_json(orient='records')
docs = json.loads(docs)

input_path = "../datasets/example_questions/nasa_spacesuit_teaching.json"
with open(input_path, 'r') as json_file:
    example_queries = json.load(json_file)
    example_queries = [data['question'] for data in example_queries[:9]]

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ../cpu_models/Meta-Llama-3-8B-Instruct.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.head

../datasets/chunked_documents/nasa_teaching_spacesuit/chunking_cks_1536_ovl_8


### Question Answering

In [3]:
import ipywidgets as widgets
from IPython.display import display

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=example_queries,
    value=example_queries[0],
    description='Example Queries:',
)

# Create a text input widget
text_input = widgets.Text(
    value='',
    placeholder='Type your own query here',
    description='Query:',
)

# Create a button widget
submit_button = widgets.Button(
    description='Submit',
)

# Create a dictionary to store the output
output_storage = {'output': None, 'ordered_docs': None}

# Create a label widget to show the status
status_label = widgets.Label(value='')

# Display the dropdown, text input, button, and status label widgets
display(dropdown)
display(text_input)
display(submit_button)
display(status_label)

# Function to handle query submission
def on_submit(button):
    query = text_input.value.strip()
    if not query:  # If text input is empty, use the dropdown value
        query = dropdown.value.strip()
    
    try:
        # Update status label to indicate processing
        status_label.value = 'Processing...'

        print(f'You selected: "{query}"')
        results, ordered_docs = rag_utils.semantic_search(query, docs, model_emb, top_k=5, verbose=False)
        n_con = 3
        contexts = [doc['text'] for doc in ordered_docs[:n_con]]
        print("___________________________________")
        print("WAIT: Running answer generation...")
        output = llm.act(query, contexts)
        print("Done..")
        
        # Store the output in the dictionary for further analysis
        output_storage['query'] = query
        output_storage['output'] = output
        output_storage['ordered_docs'] = ordered_docs
        output_storage['n_contexts'] = n_con
        # Update status label to indicate completion
        status_label.value = 'Done.'
        print("___________________________________")
        print("ANSWER: ", output)
        print("___________________________________")

    except Exception as e:
        # Handle any exceptions and update the status label
        status_label.value = 'Error occurred.'
        print(f"An error occurred: {e}")
    
    # Reset the text input widget after submission
    text_input.value = ''

# Attach the function to the button click event
submit_button.on_click(on_submit)

Dropdown(description='Example Queries:', options=('How did the Apollo program address the issue of lunar dust …

Text(value='', description='Query:', placeholder='Type your own query here')

Button(description='Submit', style=ButtonStyle())

Label(value='')

You selected: "How did the Apollo program address the issue of lunar dust contamination?"
___________________________________
WAIT: Running answer generation...


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


Done..
___________________________________
ANSWER:   The Apollo program addressed the issue of lunar dust contamination through various methods and strategies. Some of the key approaches included:

1. **Designing systems to minimize dust accumulation**: The astronauts and engineers worked together to design systems that would minimize the amount of dust that accumulated on equipment and surfaces. This included using dust-repellent materials, designing equipment with easy-to-clean surfaces, and using covers or bags to contain dust.
2. **Using cleaning methods**: The astronauts used various cleaning methods to remove dust from equipment and surfaces, including brushing, wiping with damp cloths, and using vacuum cleaners. However, the vacuum cleaners often became clogged with dust, making them ineffective.
3. **Implementing dust-containment strategies**: The astronauts used jettison bags over the legs of the lunar module to contain dust and prevent it from entering the cabin. They also us

### Visualisation

In [4]:
output = output_storage['output']
ordered_docs = output_storage['ordered_docs']
n_con = output_storage['n_contexts']

extracts = ordered_docs[:n_con]
# Example usage
from IPython.display import display, HTML
queries = [output]
num_top_returned = 5
contexts = [doc['text'] for doc in extracts]
results = rag_visualisation.find_snippet(contexts, queries, num_top_returned)
html_content = rag_visualisation.generate_interactive_html(extracts, results)
display(HTML(html_content))

/home/paul/pauld/projects/satne/satnex_llm_qa/notebooks/../rag/rag_visualisation.py:132: RuntimeWarning: invalid value encountered in scalar divide
  score_colors = " ".join([f"rgba(50,205,50,{result['scores'][idx] / max(result['scores'])})" for result in results if idx in result['top_indices']])
